In [1]:
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
random.seed(108)

Training images set

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d andrewmvd/road-sign-detection

In [ ]:
!unzip /content/road-sign-detection.zip

In [ ]:
!curl -L "https://public.roboflow.com/ds/ONxyYx8ddR?key=36n4EfSQY5" > roboflow.zip; 

In [ ]:
!unzip roboflow.zip;

In [6]:
imagesFolder = "/content/export/images/"
labelsFolder = "/content/export/labels/"

images = os.listdir(imagesFolder)
labels = os.listdir(labelsFolder)


print('total OLD images: ', len(images))
print('total OLD labels: ', len(labels))

total OLD images:  29800
total OLD labels:  29800


In [10]:
i = 222
labelPath = labelsFolder + os.listdir(labelsFolder)[i]
f = open(labelPath, "r")
txtFileContent = ""

for x in f:
  print(x)

1 0.25416666666666665 0.5433333333333333 0.08958333333333333 0.07666666666666667

1 0.2520833333333333 0.545 0.07708333333333334 0.07666666666666667


In [8]:
n = 3
replacementStr = 'jpg'

for i in range(len(labels)):

    labelPath = labelsFolder + labels[i]
    f = open(labelPath, "r")
    txtFileContent = ""
    
    for x in f:
      if((x[0]=='1' or x[0] == '2' or x[0] ==  '4' or x[0] == '6' ) and (x[1] == ' ')):
        txtFileContent += x

    f.close()
    os.remove(labelPath)

    if(txtFileContent!=""):
        f = open(labelPath, "w")
        f.write(txtFileContent)
        f.close()
    else:
        # Replace last 3 characters in labels with 'jpg'

        pathToImageBeingRemoved = imagesFolder + labels[i][:-n] + replacementStr

        os.remove(pathToImageBeingRemoved)

In [9]:
i = 2082
labels = os.listdir(labelsFolder)
labelPath = labelsFolder + labels[i]
f = open(labelPath, "r")
txtFileContent = ""

for x in f:
  print(x)

1 0.0634765625 0.5419921875 0.05078125 0.0537109375

1 0.0244140625 0.5361328125 0.046875 0.0771484375

1 0.1162109375 0.5087890625 0.048828125 0.03125

1 0.142578125 0.505859375 0.0302734375 0.03125

1 0.1728515625 0.5087890625 0.0400390625 0.0302734375

1 0.1826171875 0.49609375 0.04296875 0.0283203125

1 0.2138671875 0.4970703125 0.029296875 0.03515625

1 0.2294921875 0.52734375 0.0439453125 0.0478515625

1 0.228515625 0.525390625 0.0400390625 0.046875

1 0.26171875 0.5029296875 0.037109375 0.06640625

1 0.3017578125 0.5283203125 0.0634765625 0.0634765625

1 0.34375 0.5087890625 0.025390625 0.0302734375

1 0.3798828125 0.5048828125 0.025390625 0.033203125

4 0.419921875 0.427734375 0.0126953125 0.0283203125

4 0.46875 0.470703125 0.0107421875 0.03515625

4 0.482421875 0.466796875 0.01171875 0.0302734375

1 0.51171875 0.5087890625 0.03515625 0.0498046875

1 0.51171875 0.50390625 0.0283203125 0.03125


In [ ]:
imgPath = imagesFolder + labels[i][:-n] + replacementStr
print('labels i ', labels[i])
img = Image.open(imagesFolder+labels[i][:-n] + replacementStr)
img

In [11]:
images = os.listdir(imagesFolder)
labels = os.listdir(labelsFolder)

print('total OLD images: ', len(images))
print('total OLD labels: ', len(labels))

total OLD images:  26104
total OLD labels:  26104


In [12]:
labels = os.listdir(labelsFolder)
noOfFilesToBeRemoved = 0
for i in range(len(labels)):
        
    classesInThisFile = []
    labelPath = labelsFolder + labels[i]
    f = open(labelPath, "r")

    for x in f:
        classesInThisFile.append(x[0])

    f.close()

    if  not(('4' in classesInThisFile) or ('2' in classesInThisFile) or ('6' in classesInThisFile)):
        noOfFilesToBeRemoved+=1
        
        pathToImageBeingRemoved = imagesFolder + labels[i][:-n] + replacementStr

        os.remove(pathToImageBeingRemoved)
        os.remove(labelPath)
print('files removed: ', noOfFilesToBeRemoved)

files removed:  13510


In [13]:
!mkdir RoadSignsEdited
!mkdir RoadSignsEdited/images
!mkdir RoadSignsEdited/labels

In [14]:
import xml.etree.ElementTree as ET

In [15]:
# Function to get the data from XML Annotation
def extract_info_from_xml(xml_file):
    root = ET.parse(xml_file).getroot()
    
    # Initialise the info dict 
    info_dict = {}
    info_dict['bboxes'] = []

    # Parse the XML Tree
    for elem in root:
        # Get the file name 
        if elem.tag == "filename":
            info_dict['filename'] = elem.text
            
        # Get the image size
        elif elem.tag == "size":
            image_size = []
            for subelem in elem:
                image_size.append(int(subelem.text))
            
            info_dict['image_size'] = tuple(image_size)
        
        # Get details of the bounding box 
        elif elem.tag == "object":
            bbox = {}
            for subelem in elem:
                if subelem.tag == "name":
                    bbox["class"] = subelem.text
                    
                elif subelem.tag == "bndbox":
                    for subsubelem in subelem:
                        bbox[subsubelem.tag] = int(subsubelem.text)            
            info_dict['bboxes'].append(bbox)
    
    return info_dict

In [16]:
# Dictionary that maps class names to IDs
class_name_to_id_mapping = {"trafficlight": 5,
                           "stop": 0,
                           "speedlimit": 3,
                           "crosswalk": 1}

# Convert the info dict to the required yolo format and write it to disk
def convert_to_yolov5(info_dict):
    print_buffer = []
    
    # For each bounding box
    for b in info_dict["bboxes"]:
        try:
            class_id = class_name_to_id_mapping[b["class"]]
        except KeyError:
            print("Invalid Class. Must be one from ", class_name_to_id_mapping.keys())
        
        # Transform the bbox co-ordinates as per the format required by YOLO v5
        b_center_x = (b["xmin"] + b["xmax"]) / 2 
        b_center_y = (b["ymin"] + b["ymax"]) / 2
        b_width    = (b["xmax"] - b["xmin"])
        b_height   = (b["ymax"] - b["ymin"])
        
        # Normalise the co-ordinates by the dimensions of the image
        image_w, image_h, image_c = info_dict["image_size"]  
        b_center_x /= image_w 
        b_center_y /= image_h 
        b_width    /= image_w 
        b_height   /= image_h 
        
        #Write the bbox details to the file 
        print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, b_center_x, b_center_y, b_width, b_height))
        
    # Name of the file which we have to save 
    save_file_name = os.path.join("annotations", info_dict["filename"].replace("png", "txt"))
    
    # Save the annotation to disk
    print("\n".join(print_buffer), file= open(save_file_name, "w"))

In [17]:
# Get the annotations
annotations = [os.path.join('annotations', x) for x in os.listdir('annotations') if x[-3:] == "xml"]

# Convert and save the annotations
for ann in tqdm(annotations):
    info_dict = extract_info_from_xml(ann)
    convert_to_yolov5(info_dict)

100%|██████████| 877/877 [00:00<00:00, 5749.27it/s]


In [18]:
#remove  crosswalk as we dont need these

PathToAnnotationsInTXT = [os.path.join('annotations', x) for x in os.listdir('annotations') if x[-3:] == "txt"]

PathToAnnotationsInXML = [os.path.join('annotations', x) for x in os.listdir('annotations') if x[-3:] == "xml"]

for annotation in PathToAnnotationsInXML:
  os.remove(annotation)

pathToAnnotationsToBeRemoved = []
pathToImagesToBeRemoved = []

for annotation in PathToAnnotationsInTXT:
  f = open(annotation, "r")
  for x in f:
    if(x[0] == '1'):
      pathToAnnotationsToBeRemoved.append(annotation)
      pathToImagesToBeRemoved.append('/content/images/' + os.path.basename(annotation).replace('txt', 'png'))

print(len(pathToAnnotationsToBeRemoved))

for i in range(len(pathToImagesToBeRemoved)):
  try:
    os.remove(pathToImagesToBeRemoved[i])
    os.remove(pathToAnnotationsToBeRemoved[i])
  except:
    error = 1

200


In [19]:
imagesFolder = "/content/images"
images = os.listdir(imagesFolder)
labelsFolder = "/content/annotations"
labels = os.listdir(labelsFolder)

print('total OLD images: ', len(images))
print('total OLD labels: ', len(labels))

total OLD images:  707
total OLD labels:  707


In [20]:
def moveFilesFrom(sourceFolder, toDestFolder):
  imagesInSourceFolder = os.listdir(sourceFolder);
  for imageName in tqdm(imagesInSourceFolder):
    sourceFilePath = sourceFolder + imageName
    destFilePath = toDestFolder + imageName
    shutil.move(sourceFilePath, destFilePath)

In [ ]:
sourceFolder = ["/content/images/", "/content/annotations/"]
toDestFolder = ["/content/export/images/", "/content/export/labels/"]
for i in range(len(sourceFolder)):
  moveFilesFrom(sourceFolder[i], toDestFolder[i])

In [ ]:
os.path.basename('/sdfsf/sdfdsf/erer.txt')

In [23]:
# Read images and annotations

trainImagesPath = "/content/export/images/"
labelsPath = "/content/export/labels/"

images = [os.path.join(trainImagesPath, x) for x in os.listdir(trainImagesPath)]
annotations = [os.path.join(labelsPath, x) for x in os.listdir(labelsPath)]

images.sort()
annotations.sort()

# Split the dataset into train-valid-test splits 
train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.1, random_state = 1)
#val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)

In [24]:
!mkdir train
!mkdir train/images train/labels
!mkdir train/images/train train/images/val train/labels/train train/labels/val

In [25]:
#Utility function to move images 
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            error  = 1

# Move the splits into their folders
move_files_to_folder(train_images, 'train/images/train')
move_files_to_folder(val_images, 'train/images/val/')
#move_files_to_folder(test_images, 'images/test/')
move_files_to_folder(train_annotations, 'train/labels/train/')
move_files_to_folder(val_annotations, 'train/labels/val/')
#move_files_to_folder(test_annotations, 'annotations/test/')

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
!pip install -r /content/yolov5/requirements.txt  # install

In [27]:
imagesFolder = "/content/train/images/train"
images = os.listdir(imagesFolder)
labelsFolder = "/content/train/labels/train"
labels = os.listdir(labelsFolder)

print('total OLD images: ', len(images))
print('total OLD labels: ', len(labels))

total OLD images:  11970
total OLD labels:  11970


In [28]:
!python /content/yolov5/train.py --img 480 --cfg /content/yolov5/models/yolov5n.yaml --hyp /content/yolov5/data/hyps/hyp.scratch-low.yaml --batch 64 --epochs 30 --weights '/content/best.pt' --data data.yaml --cache  --workers 24

train: weights=/content/best.pt, cfg=/content/yolov5/models/yolov5n.yaml, data=data.yaml, hyp=/content/yolov5/data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=64, imgsz=480, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=24, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-102-ge4d8360 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, a